In [1]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
import dgl
import torch
import os
import numpy as np
import dgl.nn.pytorch as dglnn
import torch.nn.functional as F
import torch.nn as nn
import pkbar


device = "cuda"

Using backend: pytorch


In [2]:
G = dgl.load_graphs("ua_train.graph")
G =G[0][0]

In [3]:
class MovieLensNetwork(torch.nn.Module):
    
    def __init__(self , in_feats , hid_feats , out_feats, ufeats , mfeats, ratings):
        
        
        super(MovieLensNetwork , self).__init__()
        
            
        assert(in_feats == (ufeats.shape[1] , mfeats.shape[1]))
        
        self.userfeatures = nn.Parameter(ufeats)
        self.moviefeatures = nn.Parameter(mfeats)
        self.feats = nn.ParameterDict({"user" : self.userfeatures , "movie" : self.moviefeatures})

        
        print(self.userfeatures.shape)
        print(self.moviefeatures.shape)
        
        self.ratings = ratings
        
        self.userfdim , self.moviefdim = in_feats
        
        self.hid_feats = hid_feats
        self.out_feats = out_feats
        
        conv1dict , conv2dict = nn.ModuleDict() , nn.ModuleDict()
        
        for i in range(1,self.ratings + 1):
            conv1dict[str(i) + "u"] = dglnn.conv.SAGEConv( (self.userfdim , self.moviefdim) , self.hid_feats , 'mean' , activation = F.relu , bias = True , norm = F.normalize )
            conv1dict[str(i) + "m"] = dglnn.conv.SAGEConv( (self.moviefdim , self.userfdim) , self.hid_feats , 'mean' , activation = F.relu , bias = True , norm = F.normalize )
            
            conv2dict[str(i) + "u"] = dglnn.conv.SAGEConv( (self.ratings * self.hid_feats) , self.out_feats , 'mean' , activation = F.relu , bias = True)#, norm = F.normalize)
            conv2dict[str(i) + "m"] = dglnn.conv.SAGEConv( (self.ratings * self.hid_feats) , self.out_feats , 'mean' , activation = F.relu , bias = True)#, norm = F.normalize)
            
    
                
        self.conv1 = dglnn.HeteroGraphConv(conv1dict , aggregate = 'stack')
        self.conv2 = dglnn.HeteroGraphConv(conv2dict , aggregate = 'stack')
        
       
        
        self.decoders = nn.ParameterDict({
            str(i) : nn.Parameter ( torch.randn( (self.out_feats * self.ratings , self.out_feats * self.ratings) , dtype = torch.float32) \
                                   ,  requires_grad = True) for i in range(1 , self.ratings + 1)
            
        })
        
        for key in self.decoders.keys():
            nn.init.xavier_uniform_(self.decoders[key])
                
    @classmethod
    def relax(cls ,res):
        for rel in res.keys():
            sh = res[rel].shape
            res[rel] = torch.reshape(res[rel] , (sh[0] , -1,))
        return res
    
    def forward(self , G):

        #x = x.to(self.device)
        
        #convolution over the 1st layer
        res = self.conv1(G , (self.feats,self.feats))
        self.relax(res)
        
        #convolution over the 2nd layer
        res = self.conv2(G , (res , res))
        self.relax(res)
        
        probs_tensors = []
        
        
        for rating in range(1,self.ratings + 1):
            u , v = G.edges(etype = str(rating) + "u")
            u , v = u.long() , v.long()
            
            users_embeddings = torch.index_select(res["user"] , index = u , dim = 0)
            movies_embeddings = torch.index_select(res["movie"], index = v , dim = 0)
        
            n = users_embeddings.shape[0]
            d = users_embeddings.shape[1]
        
            dots_tensors = [torch.bmm( (users_embeddings@decoder).view((n , 1 , -1)) , movies_embeddings.view(n , d , -1) ).view((n , -1)) for _,decoder in self.decoders.items()]
            dots = torch.stack(dots_tensors , dim = 1).squeeze()
            probs = F.log_softmax(dots , dim = 1)
            probs_tensors.append(probs)
        

        return probs_tensors
        
        
        
        

In [4]:
DATA_PATH = "../data/ml-100k_processed/"
users_feats = np.load(os.path.join(DATA_PATH , "users.npy"))
users_feats = torch.from_numpy(users_feats).float()
movies_feats = np.load(os.path.join(DATA_PATH , "movies.npy"))
movies_feats = torch.from_numpy(movies_feats).float()
G = dgl.load_graphs(os.path.join(DATA_PATH , "ua_train.graph"))[0][0]
xo = G[("user","1u","movie")]
print(xo)

Graph(num_nodes={'user': 943, 'movie': 1681},
      num_edges={('user', '1u', 'movie'): 5567},
      metagraph=[('user', 'movie', '1u')])


In [5]:
print(G.dsttypes)
print(users_feats.shape)
print(movies_feats.shape)

['movie', 'user']
torch.Size([943, 24])
torch.Size([1681, 22])


In [8]:
G = G.to(torch.device(device))
model = MovieLensNetwork( (24 , 22) , 32 , 32 , users_feats , movies_feats , 5 )
model.to(device)

torch.Size([943, 24])
torch.Size([1681, 22])


MovieLensNetwork(
  (feats): ParameterDict(
      (movie): Parameter containing: [torch.cuda.FloatTensor of size 1681x22 (GPU 0)]
      (user): Parameter containing: [torch.cuda.FloatTensor of size 943x24 (GPU 0)]
  )
  (conv1): HeteroGraphConv(
    (mods): ModuleDict(
      (1u): SAGEConv(
        (feat_drop): Dropout(p=0.0, inplace=False)
        (fc_self): Linear(in_features=22, out_features=32, bias=True)
        (fc_neigh): Linear(in_features=24, out_features=32, bias=True)
      )
      (1m): SAGEConv(
        (feat_drop): Dropout(p=0.0, inplace=False)
        (fc_self): Linear(in_features=24, out_features=32, bias=True)
        (fc_neigh): Linear(in_features=22, out_features=32, bias=True)
      )
      (2u): SAGEConv(
        (feat_drop): Dropout(p=0.0, inplace=False)
        (fc_self): Linear(in_features=22, out_features=32, bias=True)
        (fc_neigh): Linear(in_features=24, out_features=32, bias=True)
      )
      (2m): SAGEConv(
        (feat_drop): Dropout(p=0.0, inplac

In [9]:

optimizer = torch.optim.Adam(model.parameters() , lr = 0.005)
criterion = nn.NLLLoss()

rating_types = ["1","2","3","4","5"]

epochs = 2000

kbar = pkbar.Kbar(target=epochs , width=20)

for epoch in range(epochs):
    
    model.train()
    
    loss , acc , rmse , full_cnt = 0 , 0 , 0 , 0
    
    optimizer.zero_grad()
    
    
    probs = model(G)

    
    for cls , probtensor in enumerate(probs):
        
        ypred = torch.argmax( probtensor , dim = 1 ).view(-1)
        correct = torch.sum(torch.eq(ypred , cls)).item()
        ytrue = torch.full( size = (probtensor.shape[0] , )  , fill_value = cls , dtype = torch.int64).to(device) 
        
        loss += criterion(probtensor , ytrue)
        acc += correct
        rmse += ((ytrue - ypred) ** 2).sum()
        
        
        full_cnt += ypred.shape[0]
        
    

    loss.backward()
    optimizer.step()
    

    loss /= len(probs)
    acc /= full_cnt
    rmse = rmse.float() / full_cnt
    rmse = round(rmse.item() ** 0.5 , 2)
    
    kbar.add(1 , values = [("loss",loss) , ("acc",acc) , ("rmse" , rmse)])
    
    #print("Loss : {} , Accuracy : {}".format(loss , acc))


2000/2000 [====================] - 916s 458ms/step - loss: 1.0808 - acc: 0.4673 - rmse: 1.2300
